In [1]:
import os
import cv2
import numpy as np


def generate_transformation_video(
    image,
    transform="rotation",
    output="output.mp4",
    steps=100,
    fps=30
):
    """
    Generate a video showing progressive transformation of an image.
    Supported transforms:
    rotation, scaling, translation, zoom_in, zoom_out,
    tilt, flip, blur, brightness,
    skew_horizontal, perspective, shear, stretch, compression
    """
    # Load image
    if isinstance(image, str):
        img = cv2.imread(image, cv2.IMREAD_UNCHANGED)
        if img is None:
            raise ValueError(f"Could not load image from path: {image}")
    else:
        img = image

    h, w = img.shape[:2]

    # Video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output, fourcc, fps, (w, h))

    for i in range(steps):
        progress = i / (steps - 1) if steps > 1 else 1.0

        if transform == "rotation":
            angle = progress * 360
            M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1)
            frame = cv2.warpAffine(img, M, (w, h))

        elif transform == "scaling":
            scale = 1 + 0.5 * progress
            M = cv2.getRotationMatrix2D((w//2, h//2), 0, scale)
            frame = cv2.warpAffine(img, M, (w, h))

        elif transform == "translation":
            tx = int(progress * 50)
            ty = int(progress * 50)
            M = np.float32([[1, 0, tx], [0, 1, ty]])
            frame = cv2.warpAffine(img, M, (w, h))

        elif transform == "zoom_in":
            crop = int(progress * min(h, w) // 4)
            zoom = img[crop:h-crop, crop:w-crop] if crop < h//2 and crop < w//2 else img
            frame = cv2.resize(zoom, (w, h))

        elif transform == "zoom_out":
            small = cv2.resize(img, (int(w*(1-progress*0.5)), int(h*(1-progress*0.5))))
            border_v = (h - small.shape[0]) // 2
            border_h = (w - small.shape[1]) // 2
            frame = cv2.copyMakeBorder(small, border_v, h-small.shape[0]-border_v,
                                       border_h, w-small.shape[1]-border_h,
                                       cv2.BORDER_CONSTANT, value=(0,0,0))

        elif transform == "tilt":
            pts1 = np.float32([[0,0],[w-1,0],[0,h-1]])
            pts2 = np.float32([[0,0],[w-1, int(0.3*h*progress)], [int(0.2*w*progress), h-1]])
            M = cv2.getAffineTransform(pts1, pts2)
            frame = cv2.warpAffine(img, M, (w, h))

        elif transform == "flip":
            if progress < 0.5:
                frame = img.copy()
            else:
                frame = cv2.flip(img, 1)

        elif transform == "blur":
            k = int(1 + progress * 20)
            if k % 2 == 0:
                k += 1
            frame = cv2.GaussianBlur(img, (k, k), 0)

        elif transform == "brightness":
            alpha = 1.0 + 0.5 * progress
            beta = int(50 * progress)
            frame = cv2.convertScaleAbs(img, alpha=alpha, beta=beta)

        elif transform == "skew_horizontal":
            skew_matrix = np.array([[1, 0.5*progress, 0], [0, 1, 0]], dtype=np.float32)
            frame = cv2.warpAffine(img, skew_matrix, (int(w*(1+0.5*progress)), h))

        elif transform == "perspective":
            pts1 = np.float32([[50, 50], [w-50, 50], [50, h-50], [w-50, h-50]])
            pts2 = np.float32([
                [0, 0],
                [w, 0],
                [50 + 100*progress, h],
                [w-50 - 100*progress, h]
            ])
            M = cv2.getPerspectiveTransform(pts1, pts2)
            frame = cv2.warpPerspective(img, M, (w, h))

        elif transform == "shear":
            shear_matrix = np.float32([[1, progress, 0], [0, 1, 0]])
            frame = cv2.warpAffine(img, shear_matrix, (int(w + h*progress), h))

        elif transform == "stretch":
            stretched_w = int(w * (1 + 0.5 * progress))
            stretched = cv2.resize(img, (stretched_w, h))

            if stretched_w > w:
                # Crop center
                start_x = (stretched_w - w) // 2
                frame = stretched[:, start_x:start_x + w]
            else:
                # Pad if smaller
                pad_left = (w - stretched_w) // 2
                pad_right = w - stretched_w - pad_left
                frame = cv2.copyMakeBorder(stretched, 0, 0, pad_left, pad_right,
                                        cv2.BORDER_CONSTANT, value=(0, 0, 0))


        elif transform == "compression":
            frame = cv2.resize(img, (max(1, int(w*(1-progress*0.5))), h))
            frame = cv2.copyMakeBorder(frame, 0, 0, (w-frame.shape[1])//2,
                                       w-frame.shape[1]-(w-frame.shape[1])//2,
                                       cv2.BORDER_CONSTANT, value=(0,0,0))

        else:
            raise ValueError(f"Unknown transform: {transform}")

        # Resize to match output video size
        frame_resized = cv2.resize(frame, (w, h))
        out.write(frame_resized)

    out.release()
    print(f"Saved video: {output}")


In [2]:
import os

def process_folder(input_folder, output_root="Videos_weather", steps=100, fps=30):
    """
    Process all images in a folder structure like:
    input_folder/
        corruption_name/
            severity_level/
                image1.jpg
                image2.png
    Generates videos for each transformation using `generate_transformation_video`
    Only processes images from a user-specified severity.
    """
    # Ask user for severity
    severity = input("Enter severity level to use (1-5): ").strip()
    if severity not in ["1", "2", "3", "4", "5"]:
        print("Invalid severity. Using default 3.")
        severity = "3"

    os.makedirs(output_root, exist_ok=True)

    # All supported transformations
    transforms = [
        "rotation",
        "scaling",
        "translation",
        "zoom_in",
        "zoom_out",
        "tilt",
        "flip",
        "blur",
        "brightness",
        "skew_horizontal",
        "perspective",
        "shear",
        "stretch",
        "compression"
    ]

    # Loop over all corruptions in input folder
    for corruption in os.listdir(input_folder):
        corruption_path = os.path.join(input_folder, corruption, severity)
        if not os.path.exists(corruption_path):
            continue

        for filename in os.listdir(corruption_path):
            if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                image_path = os.path.join(corruption_path, filename)
                image_name, _ = os.path.splitext(filename)

                # Folder for this image
                image_out_folder = os.path.join(output_root, image_name)
                os.makedirs(image_out_folder, exist_ok=True)

                print(f"\nProcessing {filename} ({corruption}, severity={severity})...")

                # Generate a video for each transform
                for t in transforms:
                    output_path = os.path.join(image_out_folder, f"{corruption}_{t}.mp4")
                    generate_transformation_video(
                        image_path,
                        transform=t,
                        output=output_path,
                        steps=steps,
                        fps=fps
                    )

    print(f"\nAll transformations completed. Results in: {output_root}")


Corrupted images can be generated using the notebook: https://colab.research.google.com/drive/1t8In_8EXcE_-oWNfHUEb5dqiFg6ecZGq?usp=sharing

In [9]:
input_dir = r"C:\Users\Gouth\Desktop\SummerProject\Transformations15Aerial\content\outputs_2"

process_folder(input_dir)


Processing P0645.png (brightness, severity=3)...
Saved video: Videos_weather\P0645\brightness_rotation.mp4
Saved video: Videos_weather\P0645\brightness_scaling.mp4
Saved video: Videos_weather\P0645\brightness_translation.mp4
Saved video: Videos_weather\P0645\brightness_zoom_in.mp4
Saved video: Videos_weather\P0645\brightness_zoom_out.mp4
Saved video: Videos_weather\P0645\brightness_tilt.mp4
Saved video: Videos_weather\P0645\brightness_flip.mp4
Saved video: Videos_weather\P0645\brightness_blur.mp4
Saved video: Videos_weather\P0645\brightness_brightness.mp4
Saved video: Videos_weather\P0645\brightness_skew_horizontal.mp4
Saved video: Videos_weather\P0645\brightness_perspective.mp4
Saved video: Videos_weather\P0645\brightness_shear.mp4
Saved video: Videos_weather\P0645\brightness_stretch.mp4
Saved video: Videos_weather\P0645\brightness_compression.mp4

Processing P0645.png (contrast, severity=3)...
Saved video: Videos_weather\P0645\contrast_rotation.mp4
Saved video: Videos_weather\P0645\